In [1]:
import os
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"
from vllm import LLM, SamplingParams

INFO 02-06 05:03:46 __init__.py:183] Automatically detected platform cuda.


In [2]:
llm = LLM(model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B", max_model_len=61952, tensor_parallel_size=2)

config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

INFO 02-06 05:03:51 config.py:526] This model supports multiple tasks: {'embed', 'classify', 'generate', 'score', 'reward'}. Defaulting to 'generate'.
INFO 02-06 05:03:51 config.py:1383] Defaulting to use mp for distributed inference
WARNING 02-06 05:03:51 arg_utils.py:1119] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 02-06 05:03:51 config.py:1538] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 02-06 05:03:51 llm_engine.py:232] Initializing a V0 LLM engine (v0.7.1) with config: model='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-06 05:17:58 model_runner.py:1116] Loading model weights took 7.1441 GB
(VllmWorkerProcess pid=1763452) INFO 02-06 05:17:58 model_runner.py:1116] Loading model weights took 7.1441 GB
(VllmWorkerProcess pid=1763452) INFO 02-06 05:18:01 worker.py:266] Memory profiling takes 2.28 seconds
(VllmWorkerProcess pid=1763452) INFO 02-06 05:18:01 worker.py:266] the current vLLM instance can use total_gpu_memory (44.42GiB) x gpu_memory_utilization (0.90) = 39.98GiB
(VllmWorkerProcess pid=1763452) INFO 02-06 05:18:01 worker.py:266] model weights take 7.14GiB; non_torch_memory takes 0.28GiB; PyTorch activation peak memory takes 0.19GiB; the rest of the memory reserved for KV Cache is 32.37GiB.
INFO 02-06 05:18:01 worker.py:266] Memory profiling takes 2.39 seconds
INFO 02-06 05:18:01 worker.py:266] the current vLLM instance can use total_gpu_memory (44.42GiB) x gpu_memory_utilization (0.90) = 39.98GiB
INFO 02-06 05:18:01 worker.py:266] model weights take 7.14GiB; non_torch_memory takes 0.28GiB

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:14<00:00,  2.40it/s]

INFO 02-06 05:18:19 model_runner.py:1563] Graph capturing finished in 15 secs, took 0.42 GiB
(VllmWorkerProcess pid=1763452) INFO 02-06 05:18:19 model_runner.py:1563] Graph capturing finished in 15 secs, took 0.42 GiB
INFO 02-06 05:18:19 llm_engine.py:429] init engine (profile, create kv cache, warmup model) took 20.46 seconds


In [14]:
prompt_template = """You will be provided with instructions, and a sentence in English, and your task is to segment the sentence according to the instructions. Always answer i|n the following JSON format:{{'segments':List[English]}} 

Instructions: 
1. Break down the sentence into manageable phrases that each contain enough information to be accurately interpreted. 
2. Each phrase should be no more than five words while preserving grammatical integrity and interpretability. 

English: {}"""
sent = "And in some countries -- let me pick Afghanistan -- 75 percent of the little girls don't go to school. And I don't mean that they drop out of school in the third or fourth grade -- they don't go."

In [15]:
messages = [
    {
        "role": "user",
        "content": prompt_template.format(sent)
    }
]

In [16]:
sampling_params = SamplingParams(top_k=1, max_tokens=2048)

In [17]:
output = llm.chat(messages, sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.81s/it, est. speed input: 24.11 toks/s, output: 75.94 toks/s]


In [18]:
print(output[0].outputs[0].text)

<think>
Alright, so I need to help the user segment the given sentence into manageable phrases. The instructions say each phrase should be no more than five words and still be grammatically correct and understandable. Let me read the sentence again: "And in some countries -- let me pick Afghanistan -- 75 percent of the little girls don't go to school. And I don't mean that they drop out of school in the third or fourth grade -- they don't go."

Hmm, the first part is "And in some countries -- let me pick Afghanistan --". That's a bit long. Maybe I can split it into two parts. "And in some countries" is four words, but then "-- let me pick Afghanistan --" is another part. Wait, but the user's example split it into "And in some countries -- let me pick Afghanistan" and " -- 75 percent of the little girls don't go to school." So maybe I should do the same.

Next, "And I don't mean that they drop out of school in the third or fourth grade -- they don't go." That's a long phrase. I can spli